#Prepare environment

In [ ]:
!time pip install fairseq

     |████████████████████████████████| 1.7MB 7.6MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 133kB 41.4MB/s 
     |████████████████████████████████| 112kB 42.5MB/s 
     |████████████████████████████████| 645kB 36.4MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp37-none-any.whl size=141231 sha256=2491cba9a33908c899dfb1d064d391f09b9ccbcb200bde5d0966811f780fac24
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built antlr4-python3-runtime
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

real	0m7.094s
user	0m5.816s
sys	0m0.876s


In [1]:
import os
ROOTDIR='/content/drive/MyDrive/nmt_models/CInN_2021'
os.chdir(ROOTDIR)

In [ ]:
pwd

'/content/drive/MyDrive/nmt_models/CInN_2021'

#Experiment translation unit systems

In [ ]:
DIR=os.path.join(ROOTDIR,'wovi2suben')
if not os.path.exists(DIR):
    os.mkdir(DIR)
DATDIR=os.path.join(ROOTDIR,'datasets')
SRC='wovi'
TRG='suben'

In [ ]:
!time fairseq-preprocess --source-lang $SRC --target-lang $TRG \
    --trainpref $DATDIR/train --validpref $DATDIR/dev --testpref $DATDIR/test \
    --joined-dictionary \
    --destdir $DIR/binarized
!time fairseq-train \
    $DIR/binarized \
    --arch transformer --share-all-embeddings \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 --activation-fn relu \
    --lr 0.0002 --lr-scheduler inverse_sqrt \
    --dropout 0.3 \
    --max-tokens 3200 --label-smoothing 0.1 \
    --save-dir $DIR/checkpoints --log-interval 1000 --max-update 10000 \
    --keep-interval-updates -1 --save-interval-updates 0 \
    --criterion label_smoothed_cross_entropy \
    --fp16
!time fairseq-generate \
    $DIR/binarized --gen-subset test \
    --source-lang $SRC --target-lang $TRG \
    --path $DIR/checkpoints/checkpoint_best.pt --beam 5 --nbest 1 \
    --fp16 > $DIR/result.txt

In [ ]:
def extract(result, predict,start=9):
    result = open(result, 'r').read().splitlines()
    predict = open(predict, 'w')
    select = result[start:-3:5]
    select = [s.split('\t') for s in select]
    select = [(int(s[0].split('-')[-1]), s[-1]) for s in select]
    select = sorted(select)
    for s in select:
        predict.write(s[-1] + '\n')
    predict.close()

In [ ]:
def join_pieces(filename, new_filename, type='bpe'):
    """
    function joins pieces to form words
    input: pred_trgs, list of list of pieces
    input: new_filename, str

    """
    pred_trgs = [line.split(' ') for line in open(filename, 'r').read().splitlines()]
    lines = [' '.join(line).replace('@@ ','') for line in pred_trgs]
    open(new_filename, 'w').write('\n'.join(lines))

In [ ]:
#!cat $DIR/predict.wovi | sed 's/_/ /g' > $DIR/predict.vi

In [ ]:
!chmod +x /content/drive/MyDrive/nmt_models/CInN_2021/mosesdecoder/scripts/generic/multi-bleu.perl

In [ ]:
!/content/drive/MyDrive/nmt_models/CInN_2021/mosesdecoder/scripts/generic/multi-bleu.perl -lc $DATDIR/test.$TRGX < $DIRT/predict.$TRGX

In [ ]:
import os

ROOTDIR='/content/drive/MyDrive/nmt_models/CInN_2021'
DATDIR='/content/drive/MyDrive/nmt_models/CInN_2021/datasets'

comb = []
for src in ['vsub']:
    for trg in ['en', 'suben']:
        comb.append(f'{src}2{trg}')
        comb.append(f'{trg}2{src}')

for com in comb:
    DIRT = os.path.join(ROOTDIR, com)
    print(DIRT)
    if 'predict.en' in os.listdir(DIRT):
        TRGX = 'en'
    else:
        TRGX = 'vi'
    !/content/drive/MyDrive/nmt_models/CInN_2021/mosesdecoder/scripts/generic/multi-bleu.perl -lc $DATDIR/test.$TRGX < $DIRT/predict.$TRGX


/content/drive/MyDrive/nmt_models/Unchosen_2_2021/vsub2en
BLEU = 13.77, 46.0/18.8/9.0/4.6 (BP=1.000, ratio=1.063, hyp_len=33501, ref_len=31513)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
/content/drive/MyDrive/nmt_models/Unchosen_2_2021/en2vsub
BLEU = 14.32, 45.4/20.7/10.3/5.4 (BP=0.948, ratio=0.950, hyp_len=41105, ref_len=43286)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used


Vi-En w2w: BLEU = 13.62, 45.6/18.8/9.0/4.5 (BP=1.000, ratio=1.100, hyp_len=34664, ref_len=31513)

Vi-En c2w: BLEU = 14.40, 48.2/19.9/9.6/4.8 (BP=0.992, ratio=0.992, hyp_len=31252, ref_len=31513)

Vi-En s2s: BLEU = 13.42, 48.9/19.4/9.1/4.4 (BP=0.959, ratio=0.960, hyp_len=30251, ref_len=31513)

En-Vi w2c: BLEU = 13.38, 44.7/19.9/9.6/4.9 (BP=0.938, ratio=0.940, hyp_len=40683, ref_len=43286)

En-Vi l2c: BLEU = 14.62, 46.1/21.1/10.3/5.3 (BP=0.964, ratio=0.964, hyp_len=41740, ref_len=43286)

En-Vi w2w: BLEU = 20.98, 51.1/27.1/15.3/9.1 (BP=1.000, ratio=1.001, hyp_len=43327, ref_len=43286)

En-Vi s2s: BLEU = 19.68, 53.8/28.4/15.8/9.2 (BP=0.907, ratio=0.911, hyp_len=39426, ref_len=43286)

En-Vi l2w: BLEU = 23.07, 55.0/30.5/17.7/10.8 (BP=0.969, ratio=0.969, hyp_len=41965, ref_len=43286)

suben-wovi: BLEU = 23.07, 55.0/30.5/17.7/10.8 (BP=0.969, ratio=0.969, hyp_len=41965, ref_len=43286)

wovi2suben: BLEU = 11.54, 45.4/16.6/7.3/3.5 (BP=0.982, ratio=0.982, hyp_len=30957, ref_len=31513)



In [ ]:
import os

ROOTDIR='/content/drive/MyDrive/nmt_models/CInN_2021'
DATDIR='/content/drive/MyDrive/nmt_models/CInN_2021/datasets'

comb = []
for src in ['vsub']:
    for trg in ['en', 'suben']:
        comb.append(f'{src}2{trg}')
        comb.append(f'{trg}2{src}')

for com in comb:
    DIRT = os.path.join(ROOTDIR, com)
    print(DIRT)
    if os.path.exists(DIRT):
        print(f'{src}2{trg}')
        #!/content/drive/MyDrive/nmt_models/CInN_2021/mosesdecoder/scripts/generic/multi-bleu.perl -lc $DATDIR/test.en < $DIRT/predict.en
    else:
        SRC, TRG = com.split('2')
        DIR = DIRT
        os.mkdir(DIR)
        !time fairseq-preprocess --source-lang $SRC --target-lang $TRG \
            --trainpref $DATDIR/train --validpref $DATDIR/dev --testpref $DATDIR/test \
            --joined-dictionary \
            --destdir $DIR/binarized
        !time fairseq-train \
            $DIR/binarized \
            --arch transformer --share-all-embeddings \
            --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 --activation-fn relu \
            --lr 0.0002 --lr-scheduler inverse_sqrt \
            --dropout 0.3 \
            --max-tokens 3200 --label-smoothing 0.1 \
            --save-dir $DIR/checkpoints --log-interval 1000 --max-update 10000 \
            --keep-interval-updates -1 --save-interval-updates 0 \
            --criterion label_smoothed_cross_entropy \
            --fp16
        !time fairseq-generate \
            $DIR/binarized --gen-subset test \
            --source-lang $SRC --target-lang $TRG \
            --path $DIR/checkpoints/checkpoint_best.pt --beam 5 --nbest 1 \
            --fp16 > $DIR/result.txt

/content/drive/MyDrive/nmt_models/Unchosen_2_2021/vsub2en
2021-02-24 08:53:48 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/content/drive/MyDrive/nmt_models/Unchosen_2_2021/vsub2en/binarized', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='vsub', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref='/content/

#Extract prediction from the output and evaluate prediction

In [2]:
import os

ROOTDIR='/content/drive/MyDrive/nmt_models/CInN_2021'
DATDIR='/content/drive/MyDrive/nmt_models/CInN_2021/datasets'

comb = []
for src in ['wovi', 'vi', 'subvi']:
    for trg in ['en', 'suben']:
        comb.append(f'{src}2{trg}')
        comb.append(f'{trg}2{src}')

comb = sorted(comb)

##BLEU

In [ ]:
for com in comb:
    DIRT = os.path.join(ROOTDIR, com)
    print(DIRT)
    result_file = os.path.join(DIRT, 'result.txt')
    if os.path.exists(result_file):
        _, TRG = com.split('2')
        extract(result_file, os.path.join(DIRT,f'predict.{TRG}'))
        if TRG is 'subvi':
            join_pieces(os.path.join(DIRT, 'predict.subvi'), os.path.join(DIRT, 'predict.wovi'))
        if os.path.exists(os.path.join(DIRT, 'predict.wovi')):
            !cat $DIRT/predict.wovi | sed 's/_/ /g' > $DIRT/predict.vi
        if TRG is 'suben':
            join_pieces(os.path.join(DIRT, 'predict.suben'), os.path.join(DIRT, 'predict.en'))
        print(f'-----{com}------')
        TRGX = TRG[-2:]
        !/content/drive/MyDrive/nmt_models/CInN_2021/mosesdecoder/scripts/generic/multi-bleu.perl -lc $DATDIR/test.$TRGX < $DIRT/predict.$TRGX


/content/drive/MyDrive/nmt_models/Unchosen_2_2021/en2subvi
-----en2subvi------
BLEU = 18.95, 49.8/25.4/13.8/7.8 (BP=0.986, ratio=0.986, hyp_len=42690, ref_len=43286)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
/content/drive/MyDrive/nmt_models/Unchosen_2_2021/en2vi
-----en2vi------
BLEU = 13.38, 44.7/19.9/9.6/4.9 (BP=0.938, ratio=0.940, hyp_len=40683, ref_len=43286)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores fr

##TER

###Convert to TER format

In [ ]:
def convert_ter(filename):
    """
    converts a file with filename into a format acceptable by tercom
    output will have the name = filename + ".ter"
    """
    lines = open(filename, 'r').read().splitlines()
    lines = [line + f"  ({str(i)})" for i, line in enumerate(lines)]
    ofile = open(filename + ".ter", 'w')
    for line in lines:
        ofile.write(line + '\n')
    ofile.close()

References

In [ ]:
DATDIR='datasets'
for ext in ['en', 'vi']:
    filename = f"{DATDIR}/test.{ext}"
    convert_ter(filename)

In [ ]:
!head $DATDIR/test.vi.ter

bệnh An-dai-mơ  (0)
chúng ta ai cũng có đôi lúc quên thứ này thứ nọ .  (1)
có thể bạn quên gởi thiệp mừng sinh nhật của ai đó hoặc quên trả lại sách quá hạn cho thư viện .  (2)
cuộc sống chúng ta phải có lúc quên và triệu chứng này thường hay xảy ra nhiều hơn khi con người có tuổi .  (3)
chúng ta ai cũng có đôi lúc quên thứ này thứ nọ .  (4)
có thể bạn quên gởi thiệp mừng sinh nhật của ai đó hoặc quên trả lại sách quá hạn cho thư viện .  (5)
cuộc sống chúng ta phải có lúc quên và triệu chứng này thường hay xảy ra nhiều hơn khi con người có tuổi .  (6)
nhưng bệnh An-dai-mơ , xảy ra ở một số người già , khác xa với chứng bệnh ngày nào cũng quên .  (7)
đây là bệnh thường ảnh hưởng đến não , và dần dần nó làm cho người ta khó nhớ nhiều hơn thậm chí đến những chuyện dễ dàng , cơ bản nhất , chẳng hạn như cột dây giày ra làm sao .  (8)
cuối cùng thì người bệnh có thể khó nhớ nổi tên hoặc khuôn mặt của người thân trong gia đình - hoặc thậm chí là không nhớ ra mình là ai nữa .  (9)


In [ ]:
!head $DATDIR/test.en.ter

Alzheimer disease  (0)
we all forget things once in a while .  (1)
maybe you 've forgotten to send a card for someone 's birthday or to return an overdue library book .  (2)
forgetting stuff is a part of life and it often becomes more common as people age .  (3)
we all forget things once in a while .  (4)
maybe you 've forgotten to send a card for someone 's birthday or to return an overdue library book .  (5)
forgetting stuff is a part of life and it often becomes more common as people age .  (6)
but Alzheimer disease , which affects some older people , is different from everyday forgetting .  (7)
it is a condition that permanently affects the brain , and over time , makes it harder to remember even basic stuff , like how to tie a shoe .  (8)
eventually , the person may have trouble remembering the names and faces of family members - or even who he or she is .  (9)


Hypothesis

In [ ]:
comb = []
for src in ['wovi', 'vi', 'subvi']:
    for trg in ['en', 'suben']:
        comb.append(f'{src}2{trg}')
        comb.append(f'{trg}2{src}')

comb = sorted(comb)
for com in comb:
    print(com)
    TRGX = com[-2:]
    filename = f"{com}/predict.{TRGX}"
    convert_ter(filename)

en2subvi
en2vi
en2wovi
suben2subvi
suben2vi
suben2wovi
subvi2en
subvi2suben
vi2en
vi2suben
wovi2en
wovi2suben


In [ ]:
!head en2subvi/predict.vi.ter

bệnh Kawata  (0)
tất cả chúng ta quên mọi thứ trong một lúc .  (1)
có lẽ bạn đã quên lấy thẻ cho một người nào đó để trở về một quyển sách hay một quyển sách .  (2)
quên là một phần của cuộc sống và thường trở nên phổ biến hơn khi mọi người trưởng thành .  (3)
tất cả chúng ta quên mọi thứ trong một lúc .  (4)
có lẽ bạn đã quên lấy thẻ cho một người nào đó để trở về một quyển sách hay một quyển sách .  (5)
quên là một phần của cuộc sống và thường trở nên phổ biến hơn khi mọi người trưởng thành .  (6)
nhưng bệnh Kronronen , bệnh này ảnh hưởng đến một số người lớn tuổi hơn , nhưng không phải mỗi ngày .  (7)
bệnh này là bệnh ảnh hưởng đến não , và thời gian , làm cho việc đó càng khó mà nhớ đến những thứ cơ bản hơn , chẳng hạn như việc giày dép dép dép xôn , như thế nào .  (8)
cuối cùng , người ta có thể gặp rắc rối về những cái tên và khuôn mặt của các thành viên trong gia đình - thậm chí là người ấy hay thậm chí là bé .  (9)


###Evaluate with tercom

In [ ]:
for com in comb:
    print(com)
    TRGX = com[-2:]
    !java -jar tercom.7.25.jar -r "$DATDIR"/test."$TRGX".ter -h "$com"/predict."$TRGX".ter > "$com"/ter.txt


en2subvi
en2vi
en2wovi
suben2subvi
suben2vi
suben2wovi
subvi2en
subvi2suben
vi2en
vi2suben
wovi2en
wovi2suben


In [ ]:
for com in comb:
    print(com)
    TRGX = com[-2:]
    !tail -4 "$com"/ter.txt | head -1

en2subvi
Total TER: 0.6978930832139721 (30209.0/43286.0)
en2vi
Total TER: 0.7513514762278797 (32523.0/43286.0)
en2wovi
Total TER: 0.683107702259391 (29569.0/43286.0)
suben2subvi
Total TER: 0.6525897518828259 (28248.0/43286.0)
suben2vi
Total TER: 0.7705262671533522 (33353.0/43286.0)
suben2wovi
Total TER: 0.6584576999491752 (28502.0/43286.0)
subvi2en
Total TER: 0.7151017040586425 (22535.0/31513.0)
subvi2suben
Total TER: 0.706438612636055 (22262.0/31513.0)
vi2en
Total TER: 0.708945514549551 (22341.0/31513.0)
vi2suben
Total TER: 0.740583251356583 (23338.0/31513.0)
wovi2en
Total TER: 0.7640021578396218 (24076.0/31513.0)
wovi2suben
Total TER: 0.7484530193888237 (23586.0/31513.0)


##CollGram

In [3]:
import re
from itertools import chain
from math import log, sqrt
def collgram(reference_corpus, N, learner_corpus):
    """
    computes MI score and t-score 
    input:
    reference_corpus(str) - link to the reference corpus
    N(int) - size of the reference corpus
    learner_corpus(str) - link to the learner corpus
    output:
    scores of CollGram technique: MI score(float) and t-score(float)
    """
    reference_corpus = open(reference_corpus, 'r').read()
    learner_corpus = open(learner_corpus, 'r').read()
    meaning_phrases = re.split("\n|(?<!\d)[\?\!,.](?!\d)", learner_corpus)
    bigrams = [b for l in meaning_phrases for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
    dictionary = set(chain.from_iterable(bigrams))
    counts = {token: reference_corpus.count(token) for token in dictionary}
    Os = [reference_corpus.count(f"{i} {j}") for i,j in bigrams]
    Es = [counts[i]*counts[j]/N for i,j in bigrams]
    scores = [(log(O/E, 2), (O-E)/sqrt(O)) for O,E in zip(Os, Es) if O != 0 and E != 0]
    meanMI = sum([score[0] for score in scores])/len(scores)
    mean_t_score = sum([score[1] for score in scores])/len(scores)
    return meanMI, mean_t_score

In [5]:
Ns = dict()
for ext in ['en', 'vi']:
    reference_corpus = f"{DATDIR}/train.{ext}"
    ref = open(reference_corpus, 'r').read()
    N = len(re.findall(r'\w+', ref))
    Ns[ext] = N
print(Ns)

{'en': 724231, 'vi': 1016870}


In [6]:
for com in comb:
    print(com)
    TRGX = com[-2:]
    reference_corpus = f"{DATDIR}/train.{TRGX}"
    learner_corpus = f"{com}/predict.{TRGX}"
    x = collgram(reference_corpus, Ns[TRGX], learner_corpus)
    print(x)


en2subvi
(3.129081517944966, -18.451083929716795)
en2vi
(3.7010325131848165, -16.200660012402288)
en2wovi
(3.192994583836798, -14.276648951647095)
suben2subvi
(3.0802239506053124, -16.55942610116526)
suben2vi
(3.5691579008416285, -14.543163205972936)
suben2wovi
(3.1749692415049204, -18.433742414542586)
subvi2en
(1.0984190794823483, -55.53341489092003)
subvi2suben
(0.960115998342372, -70.5831189936073)
vi2en
(1.112369208137499, -60.700725964227445)
vi2suben
(1.023936791412845, -71.66690922653581)
wovi2en
(1.1099403356901925, -59.15740899245958)
wovi2suben
(1.1056679082333498, -60.73035430479805)


#Tokenize sentences into subtokens

In [ ]:
!pip install subword-nmt

In [ ]:
!subword-nmt learn-bpe -s 10000 < $DATDIR/train.wovi > $DATDIR/vi.bpe

In [ ]:
!subword-nmt learn-bpe -s 10000 < $DATDIR/train.vi > $DATDIR/vsub.bpe

In [ ]:
!subword-nmt apply-bpe -c $DATDIR/vsub.bpe < $DATDIR/test.vi > $DATDIR/test.vsub
!subword-nmt apply-bpe -c $DATDIR/vsub.bpe < $DATDIR/dev.vi > $DATDIR/dev.vsub
!subword-nmt apply-bpe -c $DATDIR/vsub.bpe < $DATDIR/train.vi > $DATDIR/train.vsub

In [ ]:
!head -50 $DATDIR/test.vsub

bệnh An@@ -@@ da@@ i-@@ mơ
chúng ta ai cũng có đôi lúc quên thứ này thứ nọ .
có thể bạn quên gởi thiệp mừng sinh nhật của ai đó hoặc quên trả lại sách quá hạn cho thư viện .
cuộc sống chúng ta phải có lúc quên và triệu chứng này thường hay xảy ra nhiều hơn khi con người có tuổi .
chúng ta ai cũng có đôi lúc quên thứ này thứ nọ .
có thể bạn quên gởi thiệp mừng sinh nhật của ai đó hoặc quên trả lại sách quá hạn cho thư viện .
cuộc sống chúng ta phải có lúc quên và triệu chứng này thường hay xảy ra nhiều hơn khi con người có tuổi .
nhưng bệnh An@@ -@@ da@@ i-@@ mơ , xảy ra ở một số người già , khác xa với chứng bệnh ngày nào cũng quên .
đây là bệnh thường ảnh hưởng đến não , và dần dần nó làm cho người ta khó nhớ nhiều hơn thậm chí đến những chuyện dễ dàng , cơ bản nhất , chẳng hạn như cột dây giày ra làm sao .
cuối cùng thì người bệnh có thể khó nhớ nổi tên hoặc khuôn mặt của người thân trong gia đình - hoặc thậm chí là không nhớ ra mình là ai nữa .
việc này có thể rất đáng buồn đối với 

In [ ]:
!subword-nmt apply-bpe -c $DATDIR/en.bpe < $DATDIR/test.en > $DATDIR/test.suben
!subword-nmt apply-bpe -c $DATDIR/en.bpe < $DATDIR/dev.en > $DATDIR/dev.suben
!subword-nmt apply-bpe -c $DATDIR/en.bpe < $DATDIR/train.en > $DATDIR/train.suben

In [ ]:
!subword-nmt learn-bpe -s 10000 < $DATDIR/train.en > $DATDIR/en.bpe

#Extract feature of tokens

In [ ]:
!pip install stanza

     |████████████████████████████████| 286kB 7.2MB/s 


In [ ]:
import stanza
stanza.download('en')

2021-02-03 07:03:02 INFO: Downloading default packages for language: en (English)...
2021-02-03 07:03:12 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
import stanza,time
import os

def word2lemma(filename, ofilename):
    nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma', tokenize_pretokenized=True)
    isentences = open(filename, 'r').read()

    out = open(ofilename, 'w')

    doc = nlp(isentences)

    for i, isentence in enumerate(doc.sentences):
        osentence = ' '.join([word.lemma for word in isentence.words])
        out.write(osentence + '\n')
    out.close()      
    

start_time = time.time()
word2lemma(os.path.join(DATDIR,'test.en'), os.path.join(DATDIR,'test.leen'))
word2lemma(os.path.join(DATDIR,'dev.en'), os.path.join(DATDIR,'dev.leen'))
word2lemma(os.path.join(DATDIR,'train.en'), os.path.join(DATDIR,'train.leen'))
end_time = time.time()        
print(f'time in minutes: {end_time-start_time}')

2021-02-03 07:03:32 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

2021-02-03 07:03:32 INFO: Use device: gpu
2021-02-03 07:03:32 INFO: Loading: tokenize
2021-02-03 07:03:32 INFO: Loading: lemma
2021-02-03 07:03:37 INFO: Done loading processors!
2021-02-03 07:03:40 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

2021-02-03 07:03:40 INFO: Use device: gpu
2021-02-03 07:03:40 INFO: Loading: tokenize
2021-02-03 07:03:40 INFO: Loading: lemma
2021-02-03 07:03:40 INFO: Done loading processors!
2021-02-03 07:03:43 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

2021-02-03 07:03:43 INFO: Use device: gpu
2021-02-03 07:03:43 INFO: Loading: tokenize
2021-02-03 07:03:43 INFO: Load

time in minutes: 74.23542642593384


In [ ]:
!pip install stanza

     |████████████████████████████████| 286kB 9.2MB/s 


In [ ]:
import stanza
stanza.download('en')

2021-02-03 13:20:49 INFO: Downloading default packages for language: en (English)...
2021-02-03 13:21:11 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
stanza.download('vi')

2021-02-03 13:29:29 INFO: Downloading default packages for language: vi (Vietnamese)...
2021-02-03 13:29:44 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse,ner')
sentence = r"it is impossible to understand China without knowing what Fenqing is and what role they play in the society in today 's China ."
doc = nlp(sentence)

2021-02-03 13:42:25 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-02-03 13:42:25 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

2021-02-03 13:42:25 INFO: Use device: gpu
2021-02-03 13:42:25 INFO: Loading: tokenize
2021-02-03 13:42:25 INFO: Loading: pos
2021-02-03 13:42:25 INFO: Loading: lemma
2021-02-03 13:42:25 INFO: Loading: depparse
2021-02-03 13:42:26 INFO: Loading: ner
2021-02-03 13:42:26 INFO: Done loading processors!


In [ ]:
visentence = 'Fenqing là một từ tiếng Hoa mà nghĩa đen là " thanh niên phẫn nộ " .'
vinlp = stanza.Pipeline(lang='vi', processors='tokenize,pos,lemma,depparse,ner')
vidoc = vinlp(visentence)

2021-02-03 13:31:48 INFO: Loading these models for language: vi (Vietnamese):
| Processor | Package  |
------------------------
| tokenize  | vtb      |
| pos       | vtb      |
| lemma     | identity |
| depparse  | vtb      |
| ner       | default  |

2021-02-03 13:31:48 INFO: Use device: gpu
2021-02-03 13:31:48 INFO: Loading: tokenize
2021-02-03 13:31:48 INFO: Loading: pos
2021-02-03 13:31:48 INFO: Loading: lemma
2021-02-03 13:31:48 INFO: Loading: depparse
2021-02-03 13:31:49 INFO: Loading: ner
2021-02-03 13:31:49 ERROR: Cannot load model from /root/stanza_resources/vi/ner/default.pt


UnsupportedProcessorError: ignored

In [ ]:
# !pip install stanza
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse,ner')
sentence = 'đầu tư vào giáo dục là vấn đề sống còn đối với mỹ latinh và caribê .'
doc = nlp(sentence)

def __words2factors(sentence, nlp=nlp):
    doc = nlp(sentence)
    result = [' '.join([word.lemma for sent in doc.sentences for word in sent.words]),
            ' '.join([word.deprel for sent in doc.sentences for word in sent.words]),
            ' '.join([word.upos for sent in doc.sentences for word in sent.words]),
            ' '.join([word.ner for sent in doc.sentences for word in sent.words]),
            ' '.join([word.feats if word.feats else "_" for sent in doc.sentences for word in sent.words]).replace('|', ',')]
    return result

start_time = time.time()

end_time = time.time()        
epoch_mins, epoch_secs = epoch_time(start_time, end_time)
print(f'time in minutes: {epoch_mins}')

#Remove stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))
#filtered_words = [word for word in word_list if word not in stopwords.words('english')]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
from nltk.corpus import stopwords
def remove_stopwords(ifname, ofilename):
    ilines = open(ifname, 'r').read().splitlines()
    olines = [[word for word in iline.split(' ') if word not in stopwords.words('english')] for iline in ilines]
    olines = [' '.join(oline) for oline in olines]
    ofile = open(ofilename, 'w')
    for oline in olines:
        ofile.write(oline + '\n')
    ofile.close()

remove_stopwords(os.path.join(DATDIR,'test.leen'), os.path.join(DATDIR,'test.nsleen'))
remove_stopwords(os.path.join(DATDIR,'dev.leen'), os.path.join(DATDIR,'dev.nsleen'))
remove_stopwords(os.path.join(DATDIR,'train.leen'), os.path.join(DATDIR,'train.nsleen'))

Run locally

In [ ]:
folder = '/Volumes/nttrang/Writing/Complexity_2_2021/datasets'
import os
os.chdir(folder)
srctest = 'test.vi'
trgtest = 'test.en'
prde1 = 'predict.en'
pre_prde2 = 'test.wovi'
prde2 = 'predict (1).en'
prdev1 = 'predict.vi'
prdev2 = 'predict.wovi'
post_prdev2 = 'predict (1).vi'
allfiles=['test.vi', 'test.en', 'predict.en', 'test.wovi', 'predict (1).en', 'predict.vi', 'predict.wovi', 'predict (1).vi']
ps = [os.path.join(folder, file) for file in allfiles]
def catl(stop=1, start=0, length=5, ps=ps):  
    line_sets = [open(p, 'r').read().splitlines() for p in ps]

    line_sets = [line_set for line_set in zip(*line_sets) if len(line_set[0].split(' '))==length]
    for i in range(start, stop):
        print(f'----------{i}----------')
        print('\n'.join(line_sets[i]))
        print('         ')

catl(stop=50, start=0, length=32)

----------0----------
công ty đã sẵn sàng cho ra mắt chip Atom " Medfield " nó sẽ bắt đầu xuất hiện trong điện thoại thông minh và máy tính bảng vào năm tới .
the company is readying its " Medfield " Atom chips , which will begin appearing in smartphones and tablets next year .
the company was ready to launch its " Atom Atom Atom chip chip " will start to appear in smartphones and tablet computers next year .
công_ty đã sẵn_sàng cho ra_mắt chip Atom " Medfield " nó sẽ bắt_đầu xuất_hiện trong điện_thoại thông_minh và máy_tính bảng vào năm tới .
the company is ready to launch its Atom chip , which would start to appear in the smartphone and tablet tablet tablet next year .
công ty sản xuất máy tính bảng này là " chip " sẽ bắt đầu kích hoạt dòng điện thoại thông minh chạy Windows 8 và điện thoại thông minh năm nay .
công_ty này đang mở_rộng các chip " Atom " của họ , sẽ bắt_đầu biến mất trong máy_tính bảng và máy_tính bảng trong năm tới .
công ty này đang mở rộng các chip " Atom " của họ 

IndexError: ignored

In [ ]:
x = 'when evaluating a company as a possible investment , learn exactly how it makes its money .'
print(len(x.split(' ')))

17
